In [149]:
# 1. Data Preparation Script for Electricity and Economic Data
# Import necessary libraries
import pandas as pd
import requests
from io import StringIO
import numpy as np

# 1.1 Data Collection - Importing Data from URLs
# Define URLs for the datasets
data_urls = {
    'Global_Electricity_Statistics': 'https://github.com/xiaothua/dataset/raw/main/Global%20Electricity%20Statistics.csv',
    'GDP': 'https://github.com/xiaothua/dataset/raw/main/gdp.csv',
    'GDP_growth': 'https://github.com/xiaothua/dataset/raw/main/gdp_growth.csv',
    'GDP_per_capita': 'https://github.com/xiaothua/dataset/raw/main/gdp_per_capita.csv',
    'GDP_per_capita_growth': 'https://github.com/xiaothua/dataset/raw/main/gdp_per_capita_growth.csv',
    'PPP': 'https://github.com/xiaothua/dataset/raw/main/gdp_ppp.csv',
    'PPP_per_capita': 'https://github.com/xiaothua/dataset/raw/main/gdp_ppp_per_capita.csv'
}

# Load datasets into a dictionary
# Use a function to simplify data loading

def load_data_from_url(url):
    response = requests.get(url)
    response.raise_for_status()  # Ensure the request was successful
    csv_data = StringIO(response.text)
    return pd.read_csv(csv_data)

data = {key: load_data_from_url(url) for key, url in data_urls.items()}

In [150]:
# 2. Data Cleaning and Encoding

# 2.1 Electricity Data
# Delete the columns from 1980 to 1990 for electricity data, creating a new DataFrame for cleaned data. Data of 1991 will be removed later.
data_cleaned = data['Global_Electricity_Statistics'].drop(columns=[str(year) for year in range(1980, 1991)])

# Strip leading and trailing whitespaces from 'Country' and 'Features' columns
data_cleaned['Country'] = data_cleaned['Country'].str.strip()
data_cleaned['Features'] = data_cleaned['Features'].str.strip()

# Convert the data to use a MultiIndex (Country, Feature)
data_cleaned.set_index(['Country', 'Features'], inplace=True)

# Replace 'ie' and '--' with 'NA' in the entire DataFrame
data_cleaned.replace(['ie', '--'], 'NA', inplace=True)

# Convert numeric columns from object to float64 datatypes, excluding 'Region'
for column in data_cleaned.columns:
    if column != 'Region':
        data_cleaned[column] = pd.to_numeric(data_cleaned[column], errors='coerce')

# 2.2 Economic Data
# Correct Country Names in Economic Data
country_name_corrections = {
    "Bahamas, The": "Bahamas",
    'Turkey': "Türkiye",
    'Congo, Dem. Rep.': "COD",
    'Congo, Rep.': 'Republic of the Congo',
    'St. Vincent and the Grenadines': "Saint Vincent and the Grenadines",
    'St. Lucia': 'Saint Lucia',
    'Virgin Islands (U.S.)': 'U.S. Virgin Islands',
    'St. Kitts and Nevis': 'Saint Kitts and Nevis',
    'Egypt, Arab Rep.': 'Egypt',
    'Hong Kong SAR, China': 'Hong Kong',
    'Russian Federation': 'Russia',
    'Sub-Saharan Africa': 'Western Sahara',
    'Korea, Rep.': 'South Korea',
    'Iran, Islamic Rep.': 'Iran',
    'Kyrgyz Republic': 'Kyrgyzstan',
    'Syrian Arab Republic': 'Syria',
    'Yemen, Rep.': 'Yemen',
    'Macao SAR, China': 'Macau',
    'Lao PDR': 'Laos',
    # 'Myanmar': "Burma",
    'Pacific island small states': "United States Minor Outlying Islands",
    'Venezuela, RB': 'Venezuela'
}

country_name_corrections_2 = {
    'Macau': 'Macao',
    'Congo-Kinshasa': "COD",
    'Congo-Brazzaville': 'Republic of the Congo',
    'Brunei': 'Brunei Darussalam',
    'Gambia, The': 'Gambia',
    "Cote d'Ivoire": "Côte d'Ivoire",
    "U.S. Pacific Islands": "United States Minor Outlying Islands",
    "The Bahamas": "Bahamas",
    'Turkiye': "Türkiye",
    'Saint Vincent/Grenadines': "Saint Vincent and the Grenadines",
    "Burma": "Myanmar"
}

# Apply country name corrections to economic datasets only
for key in ['GDP', 'GDP_growth', 'GDP_per_capita', 'GDP_per_capita_growth', 'PPP', 'PPP_per_capita']:
    if 'Country Name' in data[key].columns:
        data[key]['Country Name'] = data[key]['Country Name'].replace(country_name_corrections)

data_cleaned.index = data_cleaned.index.set_levels(
    data_cleaned.index.levels[0].to_series().replace(country_name_corrections_2),
    level=0
)

# 2.3 Integrate Economic Data with Electricity Data
# Create new features in the cleaned electricity data to hold economic indicators
new_features = ['GDP', 'GDP_growth', 'GDP_per_capita', 'GDP_per_capita_growth', 'PPP', 'PPP_per_capita']

# Initialize new features with NaN values for each country in data_cleaned
for feature in new_features:
    data_cleaned = data_cleaned.reindex(data_cleaned.index.union(pd.MultiIndex.from_product([data_cleaned.index.levels[0], [feature]])), fill_value=np.nan)

# Populate the new features with corresponding economic data
for key in new_features:
    economic_data = data[key].set_index('Country Name')
    for country, feature in data_cleaned.index:
        if feature == key:
            try:
                for year in economic_data.columns[1:]:
                    if year in data_cleaned.columns:
                        data_cleaned.at[(country, feature), year] = economic_data.loc[country, year]
            except KeyError:
                pass

# Drop columns from 1960 to 1991, if they exist, to keep only relevant data
data_cleaned = data_cleaned.drop(columns=[str(year) for year in range(1960, 1991) if str(year) in data_cleaned.columns], errors='ignore')

# Update Region column for new economic features to match 'net generation'
for country in data_cleaned.index.levels[0]:
    net_generation_region = data_cleaned.loc[(country, 'net generation'), 'Region']
    for feature in new_features:
        data_cleaned.loc[(country, feature), 'Region'] = net_generation_region

# Remove countries with more than 9 'NA' values across all features
data_cleaned['NA_count'] = data_cleaned.isna().sum(axis=1)
countries_to_remove = data_cleaned[data_cleaned['NA_count'] >= 9].index.get_level_values(0).unique()
data_cleaned = data_cleaned[~data_cleaned.index.get_level_values(0).isin(countries_to_remove)]

# Drop the temporary 'NA_count' column
data_cleaned = data_cleaned.drop(columns=['NA_count'])

# Display the structure of the integrated data
# data_cleaned.info()
# data_cleaned.head(50)

In [151]:
# 3. Calculate more data
feature_renaming = {
    'net generation': 'Electricity: Net Generation',
    'net consumption': 'Electricity: Net Consumption',
    'imports': 'Electricity: Imports',
    'exports': 'Electricity: Exports',
    'net imports': 'Electricity: Net Imports',
    'installed capacity': 'Electricity: Installed Capacity',
    'distribution losses': 'Electricity: Distribution Losses',
    'GDP': 'Economics: GDP',
    'GDP_growth': 'Economics: GDP Growth',
    'GDP_per_capita': 'Economics: GDP Per Capita',
    'GDP_per_capita_growth': 'Economics: GDP Per Capita Growth',
    'PPP': 'Economics: PPP',
    'PPP_per_capita': 'Economics: PPP Per Capita'
}
data_cleaned = data_cleaned.rename(index=feature_renaming, level=1)

# Calculate population based on GDP and GDP per capita
population = data_cleaned.xs('Economics: GDP', level=1).drop(columns=['Region'], errors='ignore').astype(float) / data_cleaned.xs('Economics: GDP Per Capita', level=1).drop(columns=['Region'], errors='ignore').astype(float)

# Add mean and growth rate features for each electricity data feature
electricity_features = [
    'Electricity: Net Generation', 'Electricity: Net Consumption',
    'Electricity: Imports', 'Electricity: Exports',
    'Electricity: Net Imports', 'Electricity: Installed Capacity',
    'Electricity: Distribution Losses'
]

for feature in electricity_features:
    mean_feature = feature + ' Per Capita'
    growth_feature = feature + ' Growth'
    
    # Calculate mean by dividing the feature value by population for each year
    data_cleaned = data_cleaned.reindex(data_cleaned.index.union(pd.MultiIndex.from_product([data_cleaned.index.levels[0], [mean_feature, growth_feature]])), fill_value=np.nan)
    
    for country in population.index:
        for year in data_cleaned.columns:
            if year in population.columns:
                try:
                    if population.at[country, year] == 0:
                        data_cleaned.at[(country, mean_feature), year] = 0
                    else:
                        data_cleaned.at[(country, mean_feature), year] = data_cleaned.at[(country, feature), year] / population.at[country, year]
                except (KeyError, ZeroDivisionError):
                    data_cleaned.at[(country, mean_feature), year] = np.nan

    # Calculate growth rate as the percentage change from the previous year
    for year in data_cleaned.columns[1:]:
        try:
            previous_year = str(int(year) - 1)
            for country in data_cleaned.index.levels[0]:
                if data_cleaned.at[(country, feature), previous_year] == 0:
                    data_cleaned.at[(country, growth_feature), year] = 0
                else:
                    data_cleaned.at[(country, growth_feature), year] = (
                        (data_cleaned.at[(country, feature), year] - data_cleaned.at[(country, feature), previous_year])
                        / data_cleaned.at[(country, feature), previous_year]
                    ) * 100
        except (KeyError, ZeroDivisionError, ValueError):
            data_cleaned.at[(country, growth_feature), year] = np.nan

# Calculate growth rate for PPP and PPP Per Capita features
economic_features = ['Economics: PPP', 'Economics: PPP Per Capita']

for feature in economic_features:
    growth_feature = feature + ' Growth'
    
    # Initialize new growth feature with NaN values
    data_cleaned = data_cleaned.reindex(data_cleaned.index.union(pd.MultiIndex.from_product([data_cleaned.index.levels[0], [growth_feature]])), fill_value=np.nan)
    
    # Calculate growth rate as the percentage change from the previous year
    for year in data_cleaned.columns[1:]:
        try:
            previous_year = str(int(year) - 1)
            for country in data_cleaned.index.levels[0]:
                if data_cleaned.at[(country, feature), previous_year] == 0:
                    data_cleaned.at[(country, growth_feature), year] = 0
                else:
                    data_cleaned.at[(country, growth_feature), year] = (
                        (data_cleaned.at[(country, feature), year] - data_cleaned.at[(country, feature), previous_year])
                        / data_cleaned.at[(country, feature), previous_year]
                    ) * 100
        except (KeyError, ZeroDivisionError, ValueError):
            data_cleaned.at[(country, growth_feature), year] = np.nan

data_cleaned = data_cleaned.drop(columns=[str(1991), 'Region'], errors='ignore')

In [152]:
# 4. Output Final Data
# Save the cleaned data to a local CSV file for future use
data_cleaned.to_csv('data_cleaned.csv')